In [1]:
# Import des bibliothèques principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


In [2]:
# Charger les données
hr_data = pd.read_csv('general_data.csv')
survey_data = pd.read_csv('employee_survey_data.csv')
manager_data = pd.read_csv('manager_survey_data.csv')
in_time_data = pd.read_csv('in_time.csv')
out_time_data = pd.read_csv('out_time.csv')

# Aperçu des données
print(in_time_data.head())
print(out_time_data.head())

# Voir les donnees manquantes
print(hr_data.isnull().sum())
print(survey_data.isnull().sum())
print(manager_data.isnull().sum())



   Unnamed: 0  2015-01-01           2015-01-02           2015-01-05  \
0           1         NaN  2015-01-02 09:43:45  2015-01-05 10:08:48   
1           2         NaN  2015-01-02 10:15:44  2015-01-05 10:21:05   
2           3         NaN  2015-01-02 10:17:41  2015-01-05 09:50:50   
3           4         NaN  2015-01-02 10:05:06  2015-01-05 09:56:32   
4           5         NaN  2015-01-02 10:28:17  2015-01-05 09:49:58   

            2015-01-06           2015-01-07           2015-01-08  \
0  2015-01-06 09:54:26  2015-01-07 09:34:31  2015-01-08 09:51:09   
1                  NaN  2015-01-07 09:45:17  2015-01-08 10:09:04   
2  2015-01-06 10:14:13  2015-01-07 09:47:27  2015-01-08 10:03:40   
3  2015-01-06 10:11:07  2015-01-07 09:37:30  2015-01-08 10:02:08   
4  2015-01-06 09:45:28  2015-01-07 09:49:37  2015-01-08 10:19:44   

            2015-01-09           2015-01-12           2015-01-13  ...  \
0  2015-01-09 10:09:25  2015-01-12 09:42:53  2015-01-13 10:13:06  ...   
1  2015-01-09 09:4

In [3]:
# Renommer la première colonne pour qu'elle devienne "EmployeeID" dans in_time et out_time
in_time_data.rename(columns={"Unnamed: 0": 'EmployeeID'}, inplace=True)
out_time_data.rename(columns={"Unnamed: 0": 'EmployeeID'}, inplace=True)

# Supprimer les lignes avec des valeurs manquantes 
hr_data.dropna(inplace=True)
survey_data.dropna(inplace=True)
manager_data.dropna(inplace=True)

# Fusionner les données de chaque fichier
hr_data = hr_data.merge(survey_data, on='EmployeeID')
hr_data = hr_data.merge(manager_data, on='EmployeeID')
hr_data = hr_data.merge(in_time_data, on='EmployeeID')
hr_data = hr_data.merge(out_time_data, on='EmployeeID')

# Sauvegarder les données fusionnées
hr_data.to_csv('merged_data.csv', index=False)

In [6]:
import pandas as pd

# Charger les données fusionnées
hr_data = pd.read_csv('merged_data.csv')

# Créer un dictionnaire pour stocker les statistiques
statistics = []

# Calculer les statistiques pour chaque colonne
def calculate_statistics(data):
    for column in data.columns:
        col_data = data[column]
        if col_data.dtypes == 'object':  # Colonnes catégorielles
            value_counts = col_data.value_counts().to_dict()
            statistics.append({
                'Column': column,
                'Type': 'Categorical',
                'Value Counts': value_counts
            })
        else:  # Colonnes numériques
            statistics.append({
                'Column': column,
                'Type': 'Numeric',
                'Mean': col_data.mean(),
                'Median': col_data.median(),
                'StdDev': col_data.std(),
                'Min': col_data.min(),
                'Max': col_data.max(),
                'Unique Values': col_data.nunique()
            })

# Appeler la fonction de calcul des statistiques
calculate_statistics(hr_data)

# Convertir les statistiques en DataFrame pour l'export
statistics_df = pd.DataFrame(statistics)

# Sauvegarder les statistiques dans un fichier CSV
statistics_df.to_csv('statistics-data.csv', index=False)

# Afficher les statistiques dans la console
print(statistics_df)


/Users/aymane/my_env/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/aymane/my_env/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/aymane/my_env/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/aymane/my_env/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/aymane/my_env/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/aymane/my_env/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1215: Runtime

               Column         Type       Mean  Median    StdDev   Min   Max  \
0                 Age      Numeric  36.926977    36.0  9.146517  18.0  60.0   
1           Attrition  Categorical        NaN     NaN       NaN   NaN   NaN   
2      BusinessTravel  Categorical        NaN     NaN       NaN   NaN   NaN   
3          Department  Categorical        NaN     NaN       NaN   NaN   NaN   
4    DistanceFromHome      Numeric   9.197907     7.0  8.097059   1.0  29.0   
..                ...          ...        ...     ...       ...   ...   ...   
546      2015-12-25_y      Numeric        NaN     NaN       NaN   NaN   NaN   
547      2015-12-28_y  Categorical        NaN     NaN       NaN   NaN   NaN   
548      2015-12-29_y  Categorical        NaN     NaN       NaN   NaN   NaN   
549      2015-12-30_y  Categorical        NaN     NaN       NaN   NaN   NaN   
550      2015-12-31_y  Categorical        NaN     NaN       NaN   NaN   NaN   

     Unique Values                                 